In [ ]:
import sklearn.metrics
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('../src/')
from Biologging_Toolkit.applications.Rain import Rain
from Biologging_Toolkit.applications.Rain2 import Rain2
from Biologging_Toolkit.applications.Wind import Wind
import Biologging_Toolkit.utils.rain_empirical_utils as R_Utils

In [ ]:
import sklearn
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
plt.rcParams.update({
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "text.usetex": True  
    })

In [ ]:
depids = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
depids_wo293 = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_296b','ml20_313a','ml21_295a','ml21_305b'] #'ml20_293a',

path = 'D:/individus_brut/individus'
freqs = [500, 1000, 2000, 5000, 8000, 10000, 12000, 15000, 16000]
features_csv = ['downwards_mean_500', 'downwards_mean_1000', 'downwards_mean_2500',
       'downwards_mean_5000', 'downwards_mean_8000', 'downwards_mean_10000',
       'downwards_mean_12000', 'downwards_mean_15000', 'downwards_mean_16000','upwards_mean_20000','upwards_mean_5000']

In [ ]:
deps = depids[0:2] #["ml17_280a"]
deps = ['ml19_292b', 'ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
deps = ['ml17_280a', 'ml18_296a', 'ml18_294b', 'ml19_292a', 'ml19_293a']
inst = Rain(depid=deps, path=[path+f"/{depid}" for depid in deps], acoustic_path=[path+f"/{depid}/dives" for depid in deps], df_data = 'csv', frequency = 15000, wind_thresh=8, wind_speed = 'wind_speed', empirical_offset = 2, split_method = 'depid', rain_thresh = 0.2)

In [ ]:
inst.forward()

In [ ]:
inst.df.loc[(inst.df.lstm_cfosat >= 8) & (inst.df.Rain_Type_preds == 'R'), 'Rain_Type_preds'] = 'WR'

In [ ]:
df1 = inst.df

In [ ]:
inter_df = pd.concat((df1, df3))

### PER DEPID FINAL DF

In [ ]:
df = pd.DataFrame()
depids = ['ml19_292b', 'ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b', 'ml17_280a', 'ml18_296a', 'ml18_294b', 'ml19_292a', 'ml19_293a']
for depid in depids :
    inst = Rain(depid=depid, path=path+f"/{depid}", acoustic_path=path+f"/{depid}/dives", df_data = 'csv', frequency = 15000, wind_thresh=7, wind_speed = 'wind_speed', empirical_offset = 2, split_method = 'random_split', rain_thresh = 0.2)
    inst.forward()
    df = pd.concat((df, inst.df))
df.loc[(df.lstm_cfosat >= 7) & (df.Rain_Type_preds == 'R'), 'Rain_Type_preds'] = 'WR'

In [ ]:
depids = ['ml17_280a', 'ml18_296a', 'ml18_294b', 'ml19_292a', 'ml19_293a', 'ml19_292b', 'ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
precision = pd.DataFrame()
recall = pd.DataFrame()
for emp_off in [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5] :
    df = pd.DataFrame()
    for depid in depids :
        inst = Rain(depid=depid, path=path+f"/{depid}", acoustic_path=path+f"/{depid}/dives", df_data = 'csv', frequency = 15000, wind_thresh=7, wind_speed = 'wind_speed', empirical_offset = emp_off, split_method = 'random_split', rain_thresh = 0.1)
        inst.forward()
        df = pd.concat((df, inst.df))
    df.loc[(df.lstm_cfosat >= 7) & (df.Rain_Type_preds == 'R'), 'Rain_Type_preds'] = 'WR'
    rec, prec = [], []
    for cat in [['R', 'WR'], ['N']] :
        target = df.weather.copy().to_numpy()
        preds = df.Rain_Type_preds.copy().to_numpy()
        target[~np.isin(target, cat)] = 0
        target[np.isin(target, cat)] = 1
        preds[~np.isin(preds, cat)] = 0
        preds[np.isin(preds, cat)] = 1
        prec.append(sklearn.metrics.precision_score(y_true = list(target), y_pred = list(preds)))
        rec.append(sklearn.metrics.recall_score(list(target), list(preds)))
    precision[emp_off] = prec
    recall[emp_off] = rec

In [ ]:
recall.to_csv('C:/Users/grosmaan/Desktop/Thèse/Chapitre2/recall_no_rain.csv')

In [ ]:
target.sum()*0.02, (1-preds)[target == 1].sum()

---
# Detection
---

In [ ]:
inst.df["weather"].hist()

In [ ]:
R_Utils.plot_spl_rain(df)

In [ ]:
R_Utils.plot_SPL_regression(df[df.depid == 'ml17_280a'], offset = 2, save = True)

In [ ]:
R_Utils.plot_classification_spl(df)

In [ ]:
R_Utils.plot_classification_histogram(inter_df, rain = ['R'], var = ['precipitation_GPM', 'lstm_cfosat'], save = True)

In [ ]:
plot_wind_hist(intra_df,save=True)

In [ ]:
R_Utils.plot_confusion_matrix(df, normalize = "true")

In [ ]:
R_Utils.global_confusion_matrix(df1, df2, df3, df4, save = False)

### Prec/Recall Curve

In [ ]:
rslts = {}
fig, ax = plt.subplots(figsize = (6,6))
for depid in tqdm(depids):
    x = []
    y = []

    deps = [depid]
    paths = [path+f"/{depid}" for depid in deps]
    for delta in np.arange(0, 15, 0.5):
        _inst = Rain(depid=deps, path=paths, df_data = 'csv', frequency = 15000, empirical_offset=delta,
                     rain_thresh = 0.1, wind_thresh = 7, split_method = 'random_split')
        _inst.forward()
        if str(delta) not in rslts:
            rslts[str(delta)] = {"x": [], "y": []}
        _inst.df.loc[_inst.df.weather == 'WR', 'weather'] = 'N'
        _inst.df.loc[_inst.df.lstm_cfosat > 7, 'Rain_Type_preds'] = 'N'
        precision = sklearn.metrics.precision_score(_inst.df['weather'], _inst.df['Rain_Type_preds'],pos_label='R', zero_division=0)
        recall = sklearn.metrics.recall_score(_inst.df['weather'], _inst.df['Rain_Type_preds'],pos_label='R', zero_division=0)

        if recall!= 0 and precision!= 0:
                x.append(recall)
                y.append(precision)
                rslts[str(delta)]["x"].append(recall)
                rslts[str(delta)]["y"].append(precision)
    ax.plot(x, y, linestyle='--', marker='o', label=depid, alpha=0.3)

x_delta_curve = []
y_delta_curve = []
for delta in rslts.keys():
    x_delta_curve.append(np.mean(rslts[delta]["x"]))
    y_delta_curve.append(np.mean(rslts[delta]["y"]))

ax.scatter(x_delta_curve, y_delta_curve, marker="o" , color='black', label='Mean', zorder=5)
for i in range(8) :
    ax.annotate(list(rslts.keys())[i], (x_delta_curve[i] + 0.02, y_delta_curve[i]), fontsize = 12, fontweight ='bold')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(0,1.1)
ax.set_ylim(0,1.1)
ax.grid(True, linestyle="--", alpha=0.3)

ax.legend()

In [ ]:
fig.savefig('C:/Users/grosmaan/Desktop/Thèse/Chapitre2/recall_prec_depids.pdf')

---
# Estimation
---

In [ ]:
plt.scatter(df.precipitation_GPM[df.weather == 'R'], df.upwards_mean_5000[df.weather == 'R'], c = df.lstm_cfosat[df.weather == 'R'])

In [ ]:
# inst.depid_fit(bounds=((0,0),(100,100)))
inst.temporal_fit(bounds=((0,0),(100,100)))
inst.skf_fit(bounds=((0,0),(100,100)),n_splits=8)
print(inst)

In [ ]:
R_Utils.plot_rain_estimation_cumulated(inst, subset="test")

In [ ]:
R_Utils.plot_rain_estimation(inst, subset="all")

## Zoom on ml17_280a data 15-16 October 2017

In [ ]:
time_dfr = inst.df_r["begin_time"] = pd.to_datetime(inst.df_r["begin_time"], unit='s')
time_df = inst.df["begin_time"] = pd.to_datetime(inst.df["begin_time"], unit='s')

plt.figure(figsize=(20,5))
plt.plot(time_df,inst.df["precipitation_GPM"], c="lightgrey")
plt.scatter(time_dfr,inst.df_r["precipitation_GPM"], label='GPM Data')#, c="royalblue")
for split_rule in inst.popt:
    a,b = inst.popt[split_rule]
    plt.scatter(time_dfr,10**((inst.df_r["upwards_mean_15000"]+a)/b), label=f'Estimation (split:{split_rule})')#, c="lightblue")
plt.ylabel("Rain Fall (mm/h)",fontsize=15)
plt.legend()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d %b %H:%M'))
plt.xlim(pd.to_datetime(6e4 + 1.508e9, unit='s', utc=True),
         pd.to_datetime(1.8e5 + 1.508e9, unit='s', utc=True))
plt.xticks(fontsize=13)
plt.ylim(0,4)
plt.grid(True, linestyle="--", alpha=0.3)
# plt.savefig("../../data-exploration/exports/article/empEst_true_time.pdf")

plt.show()


In [ ]:
import warnings
warnings.filterwarnings('ignore')

rslts = {}
for depid in depids_wo293:
    rslts[depid]={}
    for freq in freqs :
        rslts[depid][freq] = {}

        deps =[depid]
        paths = [path+f"/{depid}" for depid in deps]

        inst = Rain(depid=deps, path=paths, df_data = 'csv', frequency = freq)
        inst.temporal_fit(bounds=((0,0),(100,100)))
        inst.skf_fit(bounds=((0,0),(100,100)),n_splits=8)
        rslts[depid][freq] = inst.rain_model_stats
        
for metric in ["mae", "rmse","r2","var","std","cc"]:
    plt.figure(figsize=(8, 5))

    for depid in rslts :
        ccs = [rslts[depid][f][f"temporal_{metric}"] for f in freqs]
        plt.plot(freqs, ccs, marker='o', linestyle='dashed', label=depid)#, color='royalblue')
    plt.xlabel('Frequency')
    plt.ylabel(metric)
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.legend(bbox_to_anchor=(1, 0.83))
    plt.show()

---
# Miscellaneous
---

---
#### Generation tableau CC Fitted
---

In [ ]:
rows = []
for depid in rslts:
    for freq in rslts[depid] :
        # print(freq, rslts[depid][freq]["skf_cc"])
        rows.append({
            "DepID": depid.replace("_", "\\_"),
            "Frequency": freq,
            "CC":rslts[depid][freq]["skf_cc"],
        })

df_long = pd.DataFrame(rows)
table_cc = df_long.pivot(index="Frequency", columns="DepID", values="CC").round(3)

latex_cc = table_cc.to_latex(
    float_format="%.3f",
    caption="$CC$ scores per depid for different frequencies",
    label="tab:cc_by_deployment_fitted",
    escape=False
)

print("=== CC TABLE ===")
print(latex_cc)

---
#### Generation tableau CC Biblio
---

In [ ]:
rslts = {}
for depid in depids_wo293 :
    rslts[depid] = {}
    for method in ["Nystuen1997","Nystuen2004","Pensieri2015","Nystuen2015"]:
        rslts[depid][method] = {}
        deps = [depid]
        inst = Rain(depid=deps, path=[path+f"/{depid}" for depid in deps], df_data = 'csv', frequency = 15000, method=method)
        inst.estimate_bibliography()
        rslts[depid][method] = inst.rain_model_stats

In [ ]:
rows = []
for depid in rslts:
    for method in rslts[depid] :
        rows.append({
            "DepID": depid.replace("_", "\\_"),
            "Method": method,
            "CC":rslts[depid][method]["biblio_cc"],
        })

df_long = pd.DataFrame(rows)
table_cc = df_long.pivot(index="Method", columns="DepID", values="CC").round(3)

latex_cc = table_cc.to_latex(
    float_format="%.3f",
    caption="$CC$ scores per depid for different frequencies",
    label="tab:cc_by_deployment_biblio",
    escape=False
)

print("=== CC TABLE ===")
print(latex_cc)